# 판단

In [39]:
import datetime
import MySQLdb
from time import sleep
import requests


stop_buffer = 0
scrap_buffer = 0
sound_buffer = 0
abnormal_buffer = 0
nothing_buffer = 0



while True :

    sleep(4)
    k = 0
    
    nowt = datetime.datetime.now()
    s8 = datetime.timedelta(seconds=8)
    s10 = datetime.timedelta(seconds=10)
    s16 = datetime.timedelta(seconds=16)
    s7 = datetime.timedelta(seconds=7)
    
    dt1 = nowt-s8
    dt2 = nowt-s16
    t1 = dt1-s10
    t2 = dt2-s7
    
    db = MySQLdb.connect("smurf1213.cafe24.com","smurf1213","1q2w3e4r!","smurf1213" )
    cursor = db.cursor()

    
#### select Raspberry table ################################################################
    cursor.execute("SELECT value from RaspberryPi \
                    WHERE  date >= '%s' and date < '%s';" % (t2, dt2))
    R_row = cursor.fetchone()
    a = 0
    b = 0
    while R_row is not None:
        a = a + R_row[0]
        b = b + 1
        R_row = cursor.fetchone()

    if a == 0:
        ave_rasp = 0
    else:
        ave_rasp = a/b
    #print ave_rasp


#### select Arduino table #####################################################################
    cursor.execute("SELECT value from Arduino \
                    WHERE  date >= '%s' and date < '%s';" % (t1,dt1))
    A_row = cursor.fetchone()

    c = 0
    d = 0
    while A_row is not None:
        c = c + A_row[0]
        d = d + 1
        A_row = cursor.fetchone()

    if c==0:
        ave_ardu = 0
    else:
        ave_ardu = c/d
    #print ave_ardu

    
##### Raspberry Buffer now time#############################################################

    cursor.execute("SELECT value from RaspberryPi \
                    WHERE  date >= '%s' and date < '%s';" % (dt2,nowt))
    BR_row = cursor.fetchone()

    e = 0
    f = 0
    while BR_row is not None:
        e = e + BR_row[0]
        f = f + 1
        BR_row = cursor.fetchone()

    if e==0:
        ave_rbuf = 0
        
    else:
        ave_rbuf = e/f
    #print ave_rbuf
    
    
##### Arduino buffer now time##########################################################

    cursor.execute("SELECT value from Arduino \
                    WHERE  date >= '%s' and date < '%s';" % (dt1,nowt))
    Ba_row = cursor.fetchone()

    g = 0
    h = 0
    while Ba_row is not None:
        g = g + Ba_row[0]
        h = h + 1
        Ba_row = cursor.fetchone()

    if g==0:
        ave_abuf = 0
    else:
        ave_abuf = g/h
    #print ave_abuf



#################################### making a decision##################################




##### Normal sound  0  ######################################################################
    if ave_rasp >= 80 and ave_ardu > 0 :
        cursor.execute("INSERT INTO decision(state) VALUES (1)")
        print "Normal"
        
##### Normal sound , No scrap   1   #########################################################
    elif ave_rasp >= 80 and ave_ardu == 0 :
        scrap_buffer = scrap_buffer + 1

        if scrap_buffer >=20:
            
            if ave_ardu == 0:
                print "Error: Unexpected error"
                cursor.execute("INSERT INTO decision(state) VALUES (2)")
                k=2
                scrap_buffer =10
            else:
                scrap_buffer = 0
                
        else:
            print "The term of transition"
            cursor.execute("INSERT INTO decision(state) VALUES (1)")

#### ABNORMAL -> Abnormal sound with scrap    2     ########################################
    elif ave_rasp < 80 and ave_rasp > -999 and ave_ardu>0:
        sound_buffer = sound_buffer + 1
        
    
        if abnormal_buffer >= 7:
        
            if ave_rbuf >=80 :
                abnormal_buffer = 0
            else:
                print "Error: one of punchers is defective"
                cursor.execute("INSERT INTO decision(state) VALUES (2)")
                k = 2
                abnormal_buffer = 4
        else:
            print "Abnormal sound is occured temporally"
            cursor.execute("INSERT INTO decision(state) VALUES (1)")
 ##### ABNORMAL -> Stopped machine       3          #########################################

    elif ave_rasp == -99999 and ave_ardu > 0 :
        
        cursor.execute("INSERT INTO decision(state) VALUES (1)")
        print "The term of transition"
        
##### ABNORMAL -> Stopped machine         4        #########################################

    elif ave_rasp == -99999 and ave_ardu == 0:
        stop_buffer = stop_buffer + 1
        
        
        if stop_buffer >= 10:
        
            if ave_rbuf == -99999 :
                print "Error: Machine has been stopped"
                cursor.execute("INSERT INTO decision(state) VALUES (3)")
                k = 3
                stop_buffer = 5
            else:
                stop_buffer = 0
                
                
        else:
            print "Punching had been stopped temporally"
            cursor.execute("INSERT INTO decision(state) VALUES (1)")
            
            
    
    
    elif ave_rasp == 0 and ave_ardu == 0:
        nothing_buffer = nothing_buffer + 1
        
        if nothing_buffer >= 10:
            if ave_rasp == 0 and ave_ardu == 0:
                print "Taking data has not being done"
                cursor.execute("INSERT INTO decision(state) VALUES (4120)")
                nothing_buffer = 5
            else:
                nothing_buffer = 0
        
    else:
        print "Exception"
        cursor.execute("INSERT INTO decision(state) VALUES (6476)")
        
    
    if k == 2: 
        who = 'Hiddenlayer'
        target_url = 'https://maker.ifttt.com/trigger/Hiddenlayer/with/key/lfpip7jTjI5b6XhPPih-G'
        r= requests.post(target_url, data={"value1" : who})
    elif k == 3:
        who = 'Hiddenlayer'
        target_url = 'https://maker.ifttt.com/trigger/Hiddenlayer/with/key/lfpip7jTjI5b6XhPPih-G'
        r= requests.post(target_url, data={"value1" : who})
        
        
        
####################

#작업 끝나면 꺼저야함

 ###########################################################################################  
    db.commit()
    db.close()

an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
an iron plate is changing or the puncher is changing
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal sound but no scrap
Normal sound but no scrap
Exception
Normal sound but no scrap
Normal sound but no scrap
Normal sound but no scrap
Normal sound but no scrap
Normal sound but no scrap
Normal sound but no scrap
Normal sound but no scrap
an iron plate is changing or the puncher is changing
Machine Error: Machine is stopped
an iron plate is changing or the puncher is changing
Normal sound 

KeyboardInterrupt: 